## MOU

## Estimation
### Moments
### MMSE
### MLE
### Bayesian

In [15]:
import os
os.chdir('/home/andrea/Work/code/neuroconn')
from MOU import MOU
from scipy.io import loadmat
import numpy as np
from scipy.stats import pearsonr

data = loadmat('/home/andrea/Work/data/AD_compact/data/alzheimers_3rd_TS.mat')
TS = data['TS']
subj = 3
ts = TS[:, :, subj].T
model = MOU(n_nodes=90)
model.fit(X=ts)
print(model.d_fit['correlation'])
C_true = model.C
ts_sim = model.simulate(T=9000)
model.fit(X=ts_sim, method='lyapunov')
print(model.d_fit['correlation'])
print(pearsonr(C_true.flatten(), model.C.flatten()))

0.724702605272
0.993945487179
(0.72723668721650825, 0.0)


0.993945487179


6858